# DigitalTWINS on FHIR - Live demo

## Introduction
The examples considered in this demo involve:
- 2 patients for Exemplar Project 1 – Biomarkers for pulmonary hypertension (note that a simplified workflow with one step/tool is used here).
- 2 patients for Exemplar Project 4 - Breast cancer reporting (6 step workflow).
- 1 patient contributed to both projects (ie in this scenario, they have both pulmonary hypertension and breast cancer).

## Definitions
- FHIR - Fast Healthcare Interoperability Resources
- FHIR Server - a server implemented according to the FHIR standard, allow users to create, update, delete, and search FHIR health data.
- Primary Measurements - a SPARC SDS dataset.
- CWL - Common Workflow Language.
- Workflow tool process - generated by executing a tool/step of a workflow.

## Learning outcomes

- Learn how to install the digitaltwins-on-fhir python client
- Learn how to find all primary measurements for a patient that have been contributed from multiple research studies. This includes:
  -  finding FHIR ImagingStudy resources and their PACS endpoints.
- Learn how to find which workflow and tool generated a specific derived measurement observation.
- Learn how to find all tools and models used by a workflow and their workflow tool processes.
- Learn how to find inputs and outputs of a given tool in a workflow.




## Installing the digitaltwins-on-fhir python client

Install package


*   pip install digitaltwins-on-fhir

In [ ]:
pip install digitaltwins-on-fhir

In [ ]:
from digitaltwins_on_fhir import Adapter
from pprint import pprint
adapter = Adapter("http://localhost:8080/fhir")
client = adapter.async_client

## Example 1: Finding all primary measurements for a patient

Step 1: Find all datasets that the patient has been involved in.

In [ ]:

# get the patient resource by uuid
patient = await client.resources("Patient").search(identifier="df0c4efd-69a6-428a-ba70-786caecfadfb").first()
target_patient = None
for p in patients:
  research_subjects = await client.resources("ResearchSubject").search(patient=p.to_reference()).fetch_all()
  dataset = []
  if len(research_subjects) > 1:
    target_patient = p
  for r in research_subjects:
    dataset.append(await r.get("study").to_resource())
  p_uuid = p.get_by_path([
        'identifier',
        {'system':'https://www.auckland.ac.nz/en/abi.html'},
        'value'
    ], '')
  print(f"Data from patient: {p_uuid} was collected from: {dataset}")



The results of the query show that there are three patients. Further queries will be performed for the patient (54f2abff-6594-11ef-917d-484d7e9beb16) whose data was collected from two studies.

Step 2: Find all primary measurements for the patient mentioned above. This will be achieved by listing the type of measurement resource type (e.g. Observation or ImagingStudy) and its code for each measurement collected in the research study.

In [ ]:
compositions = await client.resources("Composition").search(title="primary measurements",
                                                            author=target_patient.to_reference()).fetch_all()
for c in compositions:
  measurements = c.get_by_path(["section", 0, "entry"])
  dataset = await c.get('subject').to_resource()
  duuid = dataset.get_by_path([
        'identifier',
        {'system':'https://www.auckland.ac.nz/en/abi.html'},
        'value'
    ], '')
  for m in measurements:
    mr = await m.to_resource()
    code = mr.get_by_path(["code", "coding", 0, "code"])
    print(f"For research study {dataset}: {duuid}, a measurement {mr.get('resourceType')} was found with {code}.")

Step 3: List the study and series PACS endpoints for each ImagingStudy that has been peformed on the specified patient. This allows us to search for all imaging study data that has been collected in for that individual.

In [ ]:
images = await client.resources("ImagingStudy").search(subject=target_patient.to_reference()).fetch_all()

for image in images:
  composition = await client.resources("Composition").search(entry=image.to_reference()).first()
  dataset = await composition.get('subject').to_resource()
  duuid = dataset.get_by_path([
      'identifier',
      {'system':'https://www.auckland.ac.nz/en/abi.html'},
      'value'
  ], '')
  print(f"The image: {image} comes from {dataset}: {duuid}")
  image_study_endpoint = await image.get("endpoint")[0].to_resource()
  print(f"The image: {image} study PACS address is: {image_study_endpoint.get('address')}")
  for series in image.get("series"):
    series_endpoint = await series.get("endpoint")[0].to_resource()
    print(f"The image: {image} study has a PACS address of: {series_endpoint.get('address')} for one of it's series.")
  print("-----------------------------------------------------------------------------------\n")

# Example 2: Find which workflow, tool, and primary data was used to generate a specific derived measurement observation

It might of interest to find out more information regarding the provenance of a give observation e.g. called "distance to nipple" (uuid: b6b7b363-65ae-11ef-917d-484d7e9beb16-workflow-tool-6-process-002-Observation-0). For example, we could be interested in finding:
- which workflow and tool generated this observation.
- the dataset the from which the observation originated.

We can start by defining the observation of interest:

In [ ]:
ob = await client.resources("Observation").search(
            identifier="b6b7b363-65ae-11ef-917d-484d7e9beb16-workflow-tool-6-process-002-Observation-0").first()

We can find which workflow and tool generated this observation using the following steps:

In [ ]:
# Step 1: Find the workflow result composition that this observation belongs to.
composition = await client.resources("Composition").search(title="workflow tool results",
                                                            entry=ob.to_reference().reference).first()
print("Step 1: The observation belongs to this composition: ", composition)
# Step 2: Find which workflow tool process generated the composition.
workflow_tool_process = await composition.get("subject").to_resource()
print("Step 2: The composition was generated by the following workflow tool process: ", workflow_tool_process)
# Step 3: Find the related workflow from which the tool was run.
workflow = await workflow_tool_process.get("for").to_resource()
print("Step 3: The following workflow was run: ", workflow)
# Step 4: Find the tool that generated the observation.
workflow_tool = await ob.get("focus")[0].to_resource()
print("Step 4: The following tool was run: ", workflow_tool)

We can find the dataset from which the observation originated using the following steps:

In [ ]:
# Step 1: Find the research subject from whom the observation was generated.
research_subject = await workflow_tool_process.get("basedOn")[0].to_resource()
print("Step 1: The research subject is: ", research_subject)
# Step 2: Find which patient the research subject is associated with.
patient = await composition.get("author")[0].to_resource()
print("Step 2: The patient is: ", patient)
# Step 3: Find which research study the observation came from.
dataset = await research_subject.get("study").to_resource()
print("Step 3: The research study is: ", dataset)

## Example 3: Find all tools and models used by a workflow and their workflow tool processes

Lets find all workflows, and then choose a specific uuid that we can use as an example for finding all tools used by the workflow.

In [ ]:
# Get all workflows
workflows =  await client.resources("PlanDefinition").search().fetch_all()
print("All workflow resources:", workflows)
print("Workflows' uuids: ", [w.get_by_path([
        'identifier',
        {'system':'https://www.auckland.ac.nz/en/abi.html'},
        'value'
    ], '') for w in workflows ])


We can also find a workflow by searching by its `name` or `title` e.g. "breast workflow".

In [ ]:
breast_workflow = await client.resources("PlanDefinition").search(name="breast workflow").fetch_all()
print("Breast workflow:", breast_workflow)
print("uuid: ", breast_workflow[0].get_by_path([
        'identifier',
        {'system':'https://www.auckland.ac.nz/en/abi.html'},
        'value'
    ]))

We can then find all the tools that were used in this workflow including the software and/or models used in the tool (here we show how we can access the workflow from its UUID).

In [ ]:
workflow = await client.resources("PlanDefinition").search(identifier="e3b3eaa0-65ae-11ef-917d-484d7e9beb16").first()
actions = workflow.get("action")
for a in actions:
  if a.get("definitionCanonical") is None:
      continue
  resource_type, _id = a.get("definitionCanonical").split("/")
  workflow_tool = await client.reference(resource_type, _id).to_resource()
  print(f"workflow tool name: {workflow_tool.get('title')}", workflow_tool)
  print("Software and model uuids:")
  pprint(workflow_tool.get("participant"))
  print("---------------------------------")

We can also find all the workflow tool process that have been run for that particular workflow.

In [ ]:
workflow_tool_processes = await client.resources("Task").search(
            subject=workflow.to_reference().reference).fetch_all()
pprint(workflow_tool_processes)

## Example 4 Find inputs and outputs of a given tool in a workflow
Here we will specify some of the resources that we are looking for, however, all this information can be queried as shown in the previous examples.:

*   patient uuid: 54f2abff-6594-11ef-917d-484d7e9beb16
*   workflow uuid: e3b3eaa0-65ae-11ef-917d-484d7e9beb16
*   workflow tool uuid: b6b7b363-65ae-11ef-917d-484d7e9beb16
*   and research study uuid: 93d49efa-5f4e-11ef-917d-484d7e9beb16



In [ ]:
# Step 1: Find the patient.
patient = await client.resources("Patient").search(identifier="54f2abff-6594-11ef-917d-484d7e9beb16").first()
# Step 2: Find the dataset.
dataset = await client.resources("ResearchStudy").search(identifier="93d49efa-5f4e-11ef-917d-484d7e9beb16").first()
# Step 3: Find the workflow tool.
workflow_tool = await client.resources("ActivityDefinition").search(
    identifier="b6b7b363-65ae-11ef-917d-484d7e9beb16").first()
# Step 4: Find the research subject.
research_subject = await client.resources("ResearchSubject").search(patient=patient.to_reference().reference,
                                                                    study=dataset.to_reference().reference).first()
print("Research subject: ", research_subject)
# Step 5: Find the workflow tool process.
workflow_tool_process = await client.resources("Task").search(subject=workflow.to_reference(),
                                                              focus=workflow_tool.to_reference(),
                                                              based_on=research_subject.to_reference(),
                                                              owner=patient.to_reference()).first()
print("Workflow tool process: ", workflow_tool_process)

By specifying the workflow too uuid above, we have selected the tool and its execution process for the last step of the breast workflow which belongs to the specified patient and research study.

We can then find the input of the workflow tool process:

In [ ]:
inputs = workflow_tool_process.get("input")
for i in inputs:
    composition = await client.resources("Composition").search(title="primary measurements",
                                                                entry=i.get("valueReference")).first()
    print("Primary measurement's composition: ", composition)
    dataset = await composition.get("subject").to_resource()
    print("Research study: ", dataset)

We can also get all outputs of the workflow tool process:

In [ ]:
outputs = workflow_tool_process.get("output")
pprint(outputs)

We can also list all output resources:

In [ ]:
for o in outputs:
  output_resource = await o.get("valueReference").to_resource()
  print(output_resource)

Future examples:
1. From the derived data identify the consent
2. From the dervied data identify the research study
3. From the derived data identify the cohort group (future)